## Natural Gas Storage and Temperature Outcomes

This script pulls publicly available US temperature data and reported US natural gas storage activity, merges/formats that data using pandas DataFrames, and pushes the merged data to a SQL database

Data is collected from the [Energy Information Administration's API](https://www.eia.gov/opendata/)
and Climate Prediction Center's [FTP Site](ftp://ftp.cpc.ncep.noaa.gov/htdocs/degree_days/weighted/daily_data/)

**Import dependencies:**

In [1]:
import requests
import datetime
import json
import pandas as pd

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Date

##### **Establish API key and organize a dictionary of API series id's for each storage region:**
_The EIA API series IDs used in this analysis are listed [here](https://www.eia.gov/opendata/qb.php?category=1709237)_

In [2]:
eia_api_key = 'YOUR_KEY_HERE'

#this dictionary is a repository for the unique series ids for the Lower 48 and each EIA sub-region
#... the regional keys established here are also referenced to establish dictionaries for the API calls and DataFrame conversions
eia_api_series_ids = {
    'l48' : 'NG.NW2_EPG0_SWO_R48_BCF.W',
    'east' : 'NG.NW2_EPG0_SWO_R31_BCF.W',
    'midwest' : 'NG.NW2_EPG0_SWO_R32_BCF.W',
    'mountain' : 'NG.NW2_EPG0_SWO_R34_BCF.W',
    'pacific' : 'NG.NW2_EPG0_SWO_R35_BCF.W',
    'south_central' : 'NG.NW2_EPG0_SWO_R33_BCF.W',
    'salt' : 'NG.NW2_EPG0_SSO_R33_BCF.W',
    'nonsalt': 'NG.NW2_EPG0_SNO_R33_BCF.W'
}

#establishes base url for each API call
#you just need to combine with the API ID string to complete each call
base_url = f'http://api.eia.gov/series/?api_key={eia_api_key}&series_id='

#establish empty dictionaries that will be used to save API calls and DataFrames
call_dict = {}
df_dict = {}

**Create API calls:**

_using a function that iterates through the regional names established in the keys for the eia_api_series_ids dictionary_

In [3]:
def generate_call(n):
    return json.loads(requests.get(base_url + eia_api_series_ids[n]).text)

In [4]:
for n in eia_api_series_ids:
    call_dict[n] = generate_call(n)

**Convert JSON data from the API calls to pandas DataFrames**

In [5]:
def generate_df(call):
   return pd.DataFrame(call['series'][0]['data'])

In [6]:
for n in call_dict:
    df_dict[n] = generate_df(call_dict[n])

**Format the regional DataFrames**
1. _name columns_
2. _convert 'week' column to datetime format_
3. _set the index to 'week'_
4. calculate week-over-week inventory change for each week
5. _subtract 1 day from the weekly index date, to reflect the actual end date of each 'gas week'_

In [7]:
def format_df(n):
    df_dict[n].columns=['week',f'{n}_inventory']
    df_dict[n]['week'] = pd.to_datetime(df_dict[n]['week'])
    df_dict[n] = df_dict[n].sort_values('week')
    df_dict[n] = df_dict[n].set_index('week')
    df_dict[n][f'{n}_change'] = df_dict[n][f'{n}_inventory'] - df_dict[n][f'{n}_inventory'].shift(1)
    df_dict[n].index = df_dict[n].index - datetime.timedelta(days=1)

In [8]:
for n in df_dict:
    format_df(n)

In [9]:
eia_storage_data = df_dict['l48'].merge(
    df_dict['east'], left_index = True, right_index = True).merge(
    df_dict['midwest'], left_index = True, right_index = True).merge(
    df_dict['mountain'], left_index = True, right_index = True).merge(
    df_dict['pacific'], left_index = True, right_index = True).merge(
    df_dict['south_central'], left_index = True, right_index = True).merge(
    df_dict['salt'], left_index = True, right_index = True).merge(
    df_dict['nonsalt'], left_index = True, right_index = True)

### **Weather Data Pull**

**Pull HDDs**

*add note on HDD definition*

In [10]:
list_ = []
for x in range(1981,datetime.datetime.now().year+1):
    df = pd.read_csv(f'ftp://ftp.cpc.ncep.noaa.gov/htdocs/degree_days/weighted/daily_data/{x}/Population.Heating.txt',skiprows = 3, delimiter = '|').T
    df.drop(df.index[0], inplace = True)
    list_.append(df)
CPC_HDDs = pd.concat(list_)
CPC_HDDs.index = pd.to_datetime(CPC_HDDs.index)
CPC_HDDs = CPC_HDDs[9]
CPC_HDDs = pd.DataFrame(CPC_HDDs)
CPC_HDDs.columns = ['HDDs']

**Pull CDDs**

*add note on CDD definition*

In [11]:
list_ = []
for x in range(1981,datetime.datetime.now().year+1):
    df = pd.read_csv(f'ftp://ftp.cpc.ncep.noaa.gov/htdocs/degree_days/weighted/daily_data/{x}/Population.Cooling.txt',skiprows = 3, delimiter = '|').T
    df.drop(df.index[0], inplace = True)
    list_.append(df)
CPC_CDDs = pd.concat(list_)
CPC_CDDs.index = pd.to_datetime(CPC_CDDs.index)
CPC_CDDs = CPC_CDDs[9]
CPC_CDDs = pd.DataFrame(CPC_CDDs)
CPC_CDDs.columns = ['CDDs']

**Combine degree day data into a single datagrame**

...and add a Total Degree Day (TDDs) column

In [12]:
CPC_TDDs = pd.merge(CPC_HDDs,CPC_CDDs, left_index = True, right_index = True)
CPC_TDDs['TDDs'] = CPC_TDDs['HDDs'] + CPC_TDDs['CDDs']

**Resample temperature data to report sum of degree days for each week ending Thursday**

In [13]:
CPC_TDDs_Weekly = CPC_TDDs.resample('W-Thu').sum()

**Finally, combine all temperature and storage data into a single DataFrame**

... _and convert all columns to integers_

In [14]:
combined_df = pd.merge(eia_storage_data,CPC_TDDs_Weekly, left_index = True, right_index = True)

In [15]:
for n in combined_df.columns:
    combined_df[n] = pd.to_numeric(combined_df[n])

In [16]:
combined_df.tail()

,l48_inventory,l48_change,east_inventory,east_change,midwest_inventory,midwest_change,mountain_inventory,mountain_change,pacific_inventory,pacific_change,south_central_inventory,south_central_change,salt_inventory,salt_change,nonsalt_inventory,nonsalt_change,HDDs,CDDs,TDDs
2019-01-17,2370,-163.0,566,-54.0,673,-56.0,121,-6.0,185,-11.0,823,-38.0,295,-8.0,528,-29.0,199,0,199
2019-01-24,2197,-173.0,527,-39.0,606,-67.0,114,-7.0,178,-7.0,771,-52.0,278,-17.0,493,-35.0,200,0,200
2019-01-31,1960,-237.0,468,-59.0,522,-84.0,105,-9.0,172,-6.0,692,-79.0,241,-37.0,451,-42.0,229,0,229
2019-02-07,1882,-78.0,444,-24.0,492,-30.0,95,-10.0,155,-17.0,696,4.0,248,7.0,447,-4.0,160,2,162
2019-02-14,1705,-177.0,395,-49.0,436,-56.0,87,-8.0,138,-17.0,649,-47.0,224,-24.0,425,-22.0,191,1,192


### **Create the SQL database**

**Create a connection to a local SQLite database**




In [18]:
engine = create_engine('sqlite:///storage.db')

**Create the storage_weather table in the database**

In [19]:
Base.metadata.create_all(engine)

**Use a Session object to push the objects created and query the server**

In [20]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

**Finally, save the DataFrame as a local .db file**

_after creating a database schema_

In [28]:
schema_dict = {'l48_inventory' : Float(),
               'l48_change' : Float(),
               'east_inventory' : Float(),
               'east_change' : Float(),
               'midwest_inventory' : Float(),
               'midwest_change' : Float(),
               'mountain_inventory' : Float(),
               'mountain_change' : Float(),
               'pacific_inventory' : Float(),
               'pacific_change' : Float(),
               'south_central_inventory' : Float(),
               'south_central_change' : Float(),
               'salt_inventory' : Float(),
               'salt_change' : Float(),
               'nonsalt_inventory' : Float(),
               'nonsalt_change' : Float(),
               'HDDs' : Float(),
               'CDDs' : Float(),
               'TDDs' : Float()
              }

In [30]:
combined_df.to_sql('storage_weather', dtype = schema_dict, con = engine, index_label = 'week', if_exists = 'replace')